In [1]:
import cv2
import numpy as np

In [2]:
SHOW_TIMER = 1000

In [3]:
frame = cv2.imread("../frames/masked_frame.png")
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

In [4]:
cv2.imshow("frame", frame)
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("Vue", hsv_frame[:,:,2])
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows() 

In [6]:
cv2.imshow("Saturation", hsv_frame[:,:,1])
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows() 

In [7]:
cv2.imshow("Hue", hsv_frame[:,:,0])
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows() 

In [8]:
COLORS = ["blue", "red", "pink", "brown", "yellow", "green", "white", "black"]

In [9]:
MAX_BLUE = (25, 255, 255)
MIN_BLUE = (10, 0, 30)
mask_blue = cv2.inRange(hsv_frame, MIN_BLUE, MAX_BLUE) # 

In [10]:
cv2.imshow("mask blue", mask_blue) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [11]:
MAX_RED = (130, 255, 200)
MIN_RED = (110, 0, 0)
mask_red = cv2.inRange(hsv_frame, MIN_RED, MAX_RED) #

In [12]:
cv2.imshow("mask red", mask_red) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [13]:
MAX_PINK = (130, 255, 255)
MIN_PINK = (110, 0, 200)
mask_pink = cv2.inRange(hsv_frame, MIN_PINK, MAX_PINK) # TODO SELECT LARGEST PATCH


In [14]:
cv2.imshow("mask pink", mask_pink) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [15]:
MAX_BROWN = (110, 255, 150)
MIN_BROWN = (90, 0, 0)
mask_brown = cv2.inRange(hsv_frame, MIN_BROWN, MAX_BROWN) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [16]:
cv2.imshow("mask brown", mask_brown) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [17]:
MAX_YELLOW = (100, 255, 255)
MIN_YELLOW = (90, 0, 150)
mask_yellow = cv2.inRange(hsv_frame, MIN_YELLOW, MAX_YELLOW) # TODO SELECT LARGEST PATCH

In [18]:
cv2.imshow("mask yellow", mask_yellow) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [19]:
MAX_GREEN = (50, 255, 255)
MIN_GREEN = (30, 0, 0)
mask_green = cv2.inRange(hsv_frame, MIN_GREEN, MAX_GREEN) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [20]:
cv2.imshow("mask green", mask_green) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [71]:
MAX_WHITE = (90, 255, 255)
MIN_WHITE = (70, 0, 150)
mask_white = cv2.inRange(hsv_frame, MIN_WHITE, MAX_WHITE) # TODO SELECT LARGEST PATCH OR OTHER FIX

In [72]:
cv2.imshow("mask white", mask_white) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [23]:
MAX_BLACK = (50, 255, 50)
MIN_BLACK = (1, 0, 0)
mask_black = cv2.inRange(hsv_frame, MIN_BLACK, MAX_BLACK) # TODO SELECT LARGEST PATCH OR OTHER FIX


In [24]:
cv2.imshow("mask black", mask_black) 
cv2.waitKey(SHOW_TIMER)
cv2.destroyAllWindows()

In [56]:
def get_neighbours(point):
    x, y = point
    return [[x,y], [x+1,y], [x-1,y], [x,y+1], [x,y-1]]

def merge_groups(groups, ignore_groups):
    ignore_groups = []
    for current_index, current_group in enumerate(groups):
        if current_index in ignore_groups: continue
        for point in current_group:
            neighbours = get_neighbours(point)
            for index, group in enumerate(groups):
                if index == current_index: continue
                #print(group)
                exists = any(neighbour in group for neighbour in neighbours)
                if exists:
                    groups[index] = groups[index]+groups[current_index]
                    groups.pop(current_index)
                    print("merged", index, current_index, "=", len(groups))
                    print(groups)
                    return groups, True, ignore_groups
        ignore_groups += [current_index]
    print(groups)            
    return groups, False, ignore_groups
    
def get_largest_group(mask):
    candidates = np.where(mask == 255)
    
    groups = []
    for px, py in zip(candidates[0], candidates[1]):
        added = False
        neighbours = get_neighbours((px, py))
        # check if belongs to an already created group
        for index, group in enumerate(groups):
            exists = any(elem in group for elem in neighbours)
            if exists:
                groups[index] = groups[index]+[[px, py]]
                added = True
        if not added:
            print("created new")
            groups += [[[px, py]]]

    #merge groups
    ignore_groups = []
    merged=True
    while merged:
        groups, merged, ignore_groups = merge_groups(groups, ignore_groups)
    print("main", groups)
    groups = sorted(groups, key=len, reverse=True)
    if len(groups)==0: 
        print("No ball found")
        return None
    return groups[0]
    



In [57]:
COLORS

['blue', 'red', 'pink', 'brown', 'yellow', 'green', 'white', 'black']

In [82]:
raw_masks = [mask_blue, mask_red, mask_pink, mask_brown, mask_yellow, mask_green, mask_white, mask_black]
masks = []

In [83]:
"""
dilated_mask = cv2.dilate(mask_white, None, iterations=2)
largest_group = get_largest_group(dilated_mask)
new_mask = np.zeros(mask_black.shape)
for point in largest_group:
    new_mask[point[0], point[1]] = 255
cv2.imshow("test", new_mask) 
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

'\ndilated_mask = cv2.dilate(mask_white, None, iterations=2)\nlargest_group = get_largest_group(dilated_mask)\nnew_mask = np.zeros(mask_black.shape)\nfor point in largest_group:\n    new_mask[point[0], point[1]] = 255\ncv2.imshow("test", new_mask) \ncv2.waitKey(0)\ncv2.destroyAllWindows()\n'

In [84]:
# Dilate all masks and get largest patch:
for index, mask in enumerate(raw_masks):
    if index==1: 
        masks.append(mask)
        continue # Not parsing red as there are multiple red balls
    eroded_mask = cv2.erode(mask, None, iterations=2)
    dilated_mask = cv2.dilate(eroded_mask, None, iterations=2+2)
    cv2.imshow(COLORS[index], dilated_mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    largest_group = get_largest_group(dilated_mask)
    if largest_group == None: continue
    new_mask = np.zeros(mask.shape)
    for point in largest_group:
        new_mask[point[0], point[1]] = 255
    masks.append(new_mask)
    cv2.imshow(COLORS[index], new_mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

created new
created new
created new
merged 1 0 = 2
[[[270, 632], [270, 633], [270, 634], [270, 635], [270, 636], [270, 637], [270, 638], [270, 639], [270, 640], [270, 641], [270, 642], [270, 643], [270, 644], [270, 645], [270, 646], [270, 647], [271, 632], [271, 633], [271, 634], [271, 635], [271, 636], [271, 637], [271, 638], [271, 639], [271, 640], [271, 641], [271, 642], [271, 643], [271, 644], [271, 645], [271, 646], [271, 647], [272, 632], [272, 633], [272, 634], [272, 635], [272, 636], [272, 637], [272, 638], [272, 639], [272, 640], [272, 641], [272, 642], [272, 643], [272, 644], [272, 645], [272, 646], [272, 647], [273, 632], [273, 633], [273, 634], [273, 635], [273, 636], [273, 637], [273, 638], [273, 639], [273, 640], [273, 641], [273, 642], [273, 643], [273, 644], [273, 645], [273, 646], [273, 647], [274, 632], [274, 633], [274, 634], [274, 635], [274, 636], [274, 637], [274, 638], [274, 639], [274, 640], [274, 641], [274, 642], [274, 643], [274, 644], [274, 645], [274, 646],

In [92]:
for index, mask in enumerate(masks):
    cv2.imshow(COLORS[index], mask) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Now, in `masks` there are the masks for all colored balls 

In [116]:
def matches_window(frame, lower_mask, color_min_upper, color_max_upper):
    lower = lower_mask#cv2.inRange(frame, color_min_lower, color_max_lower)
    upper = cv2.inRange(frame, color_min_upper, color_max_upper)

    cv2.imshow("lower", lower) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cv2.imshow("upper", upper) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    lower_matched_x, lower_matched_y = np.where(lower==255)

    matchings = []
    for lower_candidate_x, lower_candidate_y in zip(lower_matched_x, lower_matched_y):
        if lower[lower_candidate_x, lower_candidate_y] > 0:
            if upper[lower_candidate_x-1, lower_candidate_y] == 255:
                matchings.append((lower_candidate_x-1, lower_candidate_y))
        else: print("image window out of bounds")
    return matchings
            

In [112]:
def visualize_points(image, points, color):
    # Takes a greyscale image and paints some points in color on top
    image = image.copy()# TODO
    image=np.zeros((image.shape[0], image.shape[1], 3))#.astype(int)
    #image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    for x,y in points:
        image[x,y] = color
    return image

In [127]:
highlight_masks = []

In [128]:
MIN_HIGHLIGHT = (0, 0, 245) # for the black highlight, this should be 145
MIN_HIGHLIGHT = (0, 0, 145)
MAX_HIGHLIGHT = (255, 255, 255) 
for mask in masks:
    matchings = matches_window(hsv_frame, mask, MIN_HIGHLIGHT, MAX_HIGHLIGHT)
    matchings_visualized = visualize_points(mask, matchings, (0,0,255))
    highlight_masks.append(matchings_visualized)
    cv2.imshow("matchings", matchings_visualized) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [131]:
for index, highlight in enumerate(highlight_masks):
    cv2.imshow(COLORS[index], highlight) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [130]:
cv2.imshow("matchings", highlight_masks[-1]) 
cv2.waitKey(0)
cv2.destroyAllWindows()

Now, detect the actual highlight and get their center: TODO

In [33]:
# for all ball colors (if red x 15) do
# move the window over candiate pixels or whole frame
# get hte mean of the highlight pixels
# return [(color, highlight_coord, image_mask), ()]